In [ ]:
import pandas as pd
import numpy as np


interactions = pd.read_csv("data/interactions_train.csv")
items = pd.read_csv("data/items.csv")
items.head(20)
items.info()
items.columns

print("Interactions:", interactions.shape)
print("Items:", items.shape)

interactions.head()
print(interactions.columns)



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15291 entries, 0 to 15290
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Title       15291 non-null  object
 1   Author      12638 non-null  object
 2   ISBN Valid  14568 non-null  object
 3   Publisher   15266 non-null  object
 4   Subjects    13068 non-null  object
 5   i           15291 non-null  int64 
dtypes: int64(1), object(5)
memory usage: 716.9+ KB
Interactions: (87047, 3)
Items: (15291, 6)
Index(['u', 'i', 't'], dtype='object')


In [2]:
import os
os.chdir("..")   
os.getcwd()

'd:\\Courses\\DataScience\\AI_Librarian\\Librarian_Recommender_DSML'

In [12]:
user_item_matrix = interactions.assign(interaction=1).pivot_table(
    index="u",
    columns="i",
    values="interaction",
    fill_value=0
)

user_item_matrix.head()


i,0,1,2,3,4,5,6,7,8,9,...,15280,15281,15282,15283,15284,15285,15287,15288,15289,15290
u,,,,,,,,,,,,,,,,,,,,,
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
from sklearn.metrics.pairwise import cosine_similarity

item_similarity = cosine_similarity(user_item_matrix.T)


In [14]:
item_similarity_df = pd.DataFrame(
    item_similarity,
    index=user_item_matrix.columns,
    columns=user_item_matrix.columns
)

item_similarity_df.head()



i,0,1,2,3,4,5,6,7,8,9,...,15280,15281,15282,15283,15284,15285,15287,15288,15289,15290
i,,,,,,,,,,,,,,,,,,,,,
0,1.000000,0.408248,0.333333,0.258199,0.105409,0.204124,0.258199,0.258199,0.192450,0.204124,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.408248,1.000000,0.408248,0.316228,0.129099,0.250000,0.316228,0.316228,0.235702,0.250000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.333333,0.408248,1.000000,0.258199,0.105409,0.204124,0.258199,0.258199,0.192450,0.204124,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.258199,0.316228,0.258199,1.000000,0.081650,0.158114,0.200000,0.200000,0.149071,0.158114,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.105409,0.129099,0.105409,0.081650,1.000000,0.064550,0.081650,0.081650,0.060858,0.129099,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
def recommend_items(user_id, top_k=10):
    # If user does not exist
    if user_id not in user_item_matrix.index:
        return []
    
    # Books the user has interacted with
    user_vector = user_item_matrix.loc[user_id]
    interacted_items = user_vector[user_vector > 0].index.tolist()

    # Score vector for all items
    scores = np.zeros(len(user_item_matrix.columns))

    # Aggregate similarity scores
    for item in interacted_items:
        scores += item_similarity_df[item].values

    # Remove already seen books
    for item in interacted_items:
        scores[user_item_matrix.columns.get_loc(item)] = -1e9

    # Top-k items
    top_indices = np.argsort(scores)[::-1][:top_k]
    recommended_items = user_item_matrix.columns[top_indices].tolist()
    
    return recommended_items


In [16]:
test_user = interactions["u"].sample(1).iloc[0]
print("Testing for user:", test_user)

recommend_items(test_user)


Testing for user: 2323


[3329, 3325, 4987, 3281, 3324, 3359, 3301, 88, 5820, 3326]

In [17]:
all_users = user_item_matrix.index.tolist()

recommendations = {}

for user in all_users:
    recommendations[user] = recommend_items(user)


In [20]:
submission = pd.DataFrame({
    "user_id": list(recommendations.keys()),
    "recommendation": [" ".join(map(str, recs)) for recs in recommendations.values()]
})


In [21]:
submission.to_csv("submission.csv", index=False)
